# CE7455 Assignment 2

Name: PENG HONGYI <br>
Matric No: G2105029E

## Quetion One (i)

__Named Entity Recognition__ (NER) as an important task in NLP, attemps to classify predefined entities in a sentence. In our assignment, we use _eng.train_ for training, _eng.testa_ for validation and _eng.testb_ for testing.  

A sentence in the dataset is presented below, where the first columns is the input word and the last column is the output tag. The dataset contains four different types of predefined entities: PERSON, LOCATION, ORGANIZATION, and MISC. As shown in the fourth column, the fourth column contains the groudtruth entity name and the BIO tag, seperated by '-'.

| Word    |     |      | Tag    |
|---------|-----|------|--------|
| EU      | NNP | I-NP | I-ORG  |
| rejects | VBZ | I-VP | O      |
| German  | JJ  | I-NP | I-MISC |
| call    | NN  | I-NP | O      |
| to      | TO  | I-VP | O      |
| boycott | VB  | I-VP | O      |
| British | JJ  | I-NP | I-MISC |
| lamb    | NN  | I-NP | O      |
| .       | .   | O    | O      |

## Question one (ii)


There are 5 preprocessing steps in the code provided:
* Replacing all the digit with 0.
* Convert BIO tagging to BIOES tagging.
* Generate words mapping.
* Generate tag mapping.
* Generate chracter mapping.
Mappings here are dictionaries that assign an integer id to every unique word, character and tag. After the proprocessing step, we found: 

> Found 17493 unique words (203621 in total) 
Found 75 unique characters 
Found 19 unique named entity tags

The preprocessed dataset is stored in _data/mapping.pkl_. To save time, we will directly processed data.

## Question one (iii)